In [1]:
import pickle
import pandas as pd
import numpy as np

# create dataset for best picture

In [2]:
oscar_df = pd.read_csv("data/the_oscar_award.csv")
oscar_df = oscar_df[oscar_df['year_ceremony'] >= 1937]
oscar_df['category'] = oscar_df['category'].str.lower() 
bp_mask = oscar_df['category'].str.contains('best|outstanding (motion)? picture')

c:\Users\Nick\Anaconda3\envs\NN\lib\site-packages\pandas\core\strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


## Add Nominations and win count

In [8]:
#create all masks
actor_mask = oscar_df['category'].str.contains('actor( in a leading role)?$')
actress_mask = oscar_df['category'].str.contains('actress( in a leading role)?$')
director_mask = oscar_df['category'].str.contains('directing')
writer_mask = oscar_df['category'].str.contains('writing.*screenplay')
sup_actor_mask = oscar_df['category'].str.contains('actor in a supporting role')
sup_actress_mask = oscar_df['category'].str.contains('actress in a supporting role')

mask_dct = {
    'actor': actor_mask,
    'actress': actress_mask,
    'director': director_mask,
    'writer': writer_mask,
    'sup_actor': sup_actor_mask,
    'sup_actress': sup_actress_mask,
    'best_picture': bp_mask,
}

for k,v in mask_dct.items():
    oscar_df['category'] = np.where(v,k,oscar_df['category'])



big5_mask = actor_mask | actress_mask | bp_mask | director_mask | writer_mask
atl_mask = big5_mask | sup_actor_mask | sup_actress_mask

c:\Users\Nick\Anaconda3\envs\NN\lib\site-packages\pandas\core\strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [4]:
noms = oscar_df.groupby(['year_film', 'film']).size().rename('noms')
wins = oscar_df[oscar_df['winner']==True & ~bp_mask].groupby(['year_film', 'film'])['category'].nunique().rename('wins')

big5_noms = oscar_df[big5_mask].groupby(['year_film', 'film'])['category'].nunique().rename('big5_noms')
big5_wins = oscar_df[~bp_mask & big5_mask & oscar_df['winner']==True].groupby(['year_film', 'film'])['category'].nunique().rename('big5_wins')

atl_noms = oscar_df[atl_mask].groupby(['year_film', 'film'])['category'].nunique().rename('atl_noms')
atl_wins = oscar_df[~bp_mask & atl_mask & oscar_df['winner']==True].groupby(['year_film', 'film'])['category'].nunique().rename('atl_wins')

for ser in (noms,wins,big5_noms,big5_wins,atl_noms,atl_wins):
    oscar_df = pd.merge(oscar_df,ser,left_on=['year_film', 'film'],right_index=True,how='left')

oscar_df[['wins','big5_wins','atl_wins']] = oscar_df[['wins','big5_wins','atl_wins']].fillna(0)


In [9]:
oscar_df[atl_mask & (oscar_df['year_ceremony'] == 2022)]

,year_film,year_ceremony,ceremony,category,name,film,winner,noms,wins,big5_noms,big5_wins,atl_noms,atl_wins
10515,2021,2022,94,actor,Javier Bardem,Being the Ricardos,False,3.0,0.0,2.0,0.0,3.0,0.0
10516,2021,2022,94,actor,Benedict Cumberbatch,The Power of the Dog,False,12.0,2.0,4.0,1.0,6.0,1.0
10517,2021,2022,94,actor,Andrew Garfield,"tick, tick...BOOM!",False,2.0,0.0,1.0,0.0,1.0,0.0
10518,2021,2022,94,actor,Will Smith,King Richard,True,6.0,2.0,3.0,1.0,4.0,1.0
10519,2021,2022,94,actor,Denzel Washington,The Tragedy of Macbeth,False,3.0,0.0,1.0,0.0,1.0,0.0
10520,2021,2022,94,sup_actor,Ciarán Hinds,Belfast,False,7.0,2.0,3.0,1.0,5.0,1.0
10521,2021,2022,94,sup_actor,Troy Kotsur,CODA,True,3.0,2.0,2.0,1.0,3.0,2.0
10522,2021,2022,94,sup_actor,Jesse Plemons,The Power of the Dog,False,12.0,2.0,4.0,1.0,6.0,1.0
10523,2021,2022,94,sup_actor,J.K. Simmons,Being the Ricardos,False,3.0,0.0,2.0,0.0,3.0,0.0
10524,2021,2022,94,sup_actor,Kodi Smit-McPhee,The Power of the Dog,False,12.0,2.0,4.0,1.0,6.0,1.0


In [98]:
oscar_df[writer_mask].groupby(['year_ceremony','film'])['category'].nunique().sort_values()

year_ceremony  film                       
1937           After the Thin Man             1
1995           Forrest Gump                   1
               Four Weddings and a Funeral    1
               Heavenly Creatures             1
               Nobody's Fool                  1
                                             ..
1968           Bonnie and Clyde               1
               Cool Hand Luke                 1
               Divorce American Style         1
1965           The Organizer                  1
2023           Women Talking                  1
Name: category, Length: 845, dtype: int64

In [67]:
len(oscar_df[actor_mask]['year_ceremony'].unique())

95

In [66]:
oscar_df[oscar_df['year_ceremony'] == 1934]

,year_film,year_ceremony,ceremony,category,name,film,winner,nominations,wins


In [3]:
tmdb_df = pd.read_csv('data/tmdb.csv')
tmdb_df

,Unnamed: 0,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count
0,The Noose_1927,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,The Last Command_1927,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A Ship Comes In_1927,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7th Heaven_1927,False,/qRDUrxnMjLGXBqooHvfT9zzICZ0.jpg,"[18, 10749]",82474.0,en,7th Heaven,A dejected Parisian sewer worker feels his pra...,5.476,/3ETXMu5PrcVMRJpfN2Z3MwcfXzy.jpg,1927-09-10,7th Heaven,False,7.4,77.0
4,Sadie Thompson_1927,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5184,Ivalu_2022,False,/572w5U3T7CiyAswyshiS48vO7uR.jpg,[18],1042171.0,kl,Ivalu,Ivalu is gone. Her little sister is desperate ...,1.814,/qxu1Rv9BSTMlVUKrQy2tPWEFRv9.jpg,2022-01-01,Ivalu,False,0.0,0.0
5185,Le Pupille_2022,False,/aHkjDwtkPkuc88X7FBQuN1EMjis.jpg,"[10751, 35]",974586.0,it,Le pupille,A facetious coming-of-age fable that ends with...,8.162,/tuee2So5p4zWbEznpL8EKRT4cre.jpg,2022-05-27,Le Pupille,False,6.9,21.0
5186,Night Ride_2022,False,NaN,[18],1043141.0,hr,Noćna vožnja,"Through a sequence of dreams and nightmares, D...",1.473,/keukc9pxXxGOKX8l40x4WJ5pydl.jpg,2022-10-26,Night Ride,False,0.0,0.0
5187,The Red Suitcase_2022,False,/lF1bmtWOfwqlWcu7ybJYvdqzQPD.jpg,[18],1032734.0,fa,La valise rouge,A veiled 16 year old Iranian teenager is terri...,3.487,/erJFQ6iVz73HoDG99GcwNNQN5up.jpg,2022-09-28,The Red Suitcase,False,0.0,0.0
